In [1]:
import glob
import os as os
import re as re
import pandas as pd

In [2]:
from simulation import *
from summarize import *
from aesthetics import *
from plot import *

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
mpl.rc('text', usetex=True)
cmap = sns.color_palette("Paired", 10)
prepare_plot()

# Scan concentrations

In [10]:
calculation = False

The general strategy is:

1. Specify the directory
1. Get a list of files and exclude a few that aren't rotatable
1. For each concentration, get the directional flux $J$, the intersurface flux, the reciprocating flux $J_R$, the maximum power $P_\text{max}$, and the load at the maximum power $\tau_\text{stall}/2$
1. Add these to a growing data frame
1. Save the pickle.

## PKA

In [11]:
pka_dir = './md-data/pka-md-data/'
pka_unbound_files = sorted(glob.glob(pka_dir + 'apo/' + '*'))
names = []
for file in range(len(pka_unbound_files)):
    name = os.path.splitext(os.path.basename(pka_unbound_files[file]))[0]
    name = re.search('^[^_]*', name).group(0)        
    if re.search('omega*', name):
        continue
    if re.search('chi3ASN*', name):
        continue
    if re.search('chi5LYS*', name):
        continue
    # This one is missing in the PKA data set.
    if re.search('chi1CYM185', name):
        continue
    names.append(name)

In [12]:
df = pd.DataFrame()
if calculation:
    for concentration in np.arange(-6, 0, 0.1):
        for name in names:
            directional_flux, intersurface_flux, driven_flux = summarize_fluxes(name, concentration=10**concentration,
                                                                               data_source='pka_md_data') 
            if directional_flux < 0:
                negative=True
            else:
                negative=False
            max_power, max_load = summarize_power_and_load(name, concentration=10**concentration, negative=negative, 
                                                           debug=False, data_source='pka_md_data')
            df = df.append(pd.DataFrame({'Concentration': concentration,
                                    'Directional flux': directional_flux,
                                    'Intersurface flux': intersurface_flux,
                                    'Driven flux': driven_flux,
                                    'File': name,
                                    'Max power': max_power,
                                    'Max load' : max_load,
                                    'ResID': re.match('.*?([0-9]+)$', name).group(1),
                                    }, index=[0]), ignore_index=True)

    df.to_pickle('pka-concentration-scan.pickle')